In [3]:
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56438 sha256=713a6ce0ecc85fe43d9d0860c50bd0a795ef3a4204ac736f9fe6f6f85df5617d
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [4]:
import numpy as np
import re
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.models import Model, Sequential
from keras.optimizers import RMSprop
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback
import np_utils
from keras.models import load_model, save_model
from tensorflow.keras.optimizers import Adam
import matplotlib as plt

In [5]:
load_saved_model = False
train_model = True
seq_length = 25
filename = "/content/10-million-password-list-top-100000.txt"


In [6]:
with open(filename, encoding='utf-8-sig') as f:
    text = f.read().strip()

chars = sorted(list(set(text)))
total_chars = len(chars)
print("Unique characters in the text:", total_chars)

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


Unique characters in the text: 83


In [7]:
X = np.zeros((len(text) - seq_length, seq_length, total_chars), dtype=bool)
y = np.zeros((len(text) - seq_length, total_chars), dtype=bool)

for i in range(len(text) - seq_length):
    for t, char in enumerate(text[i:i + seq_length]):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[text[i + seq_length]]] = 1


In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, diversity):
    start_index = np.random.randint(0, len(text) - seq_length - 1)
    sentence = text[start_index: start_index + seq_length]
    generated = sentence
    for i in range(length):
        x_pred = np.zeros((1, seq_length, total_chars))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char
    return generated


In [7]:
if load_saved_model:
    model = load_model('/content/model-saveLSTM.h5')
else:
    model = Sequential()
    model.add(LSTM(256, input_shape=(seq_length, total_chars), return_sequences=True))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(total_chars, activation='softmax'))
    optimizer = Adam(learning_rate=0.002)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [9]:
#cliping gradient

if load_saved_model:
    model = load_model('/content/model-saveLSTM.h5')
else:
    model = Sequential()
    model.add(LSTM(256, input_shape=(seq_length, total_chars), return_sequences=True))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(total_chars, activation='softmax'))

    # Add gradient clipping to the optimizer
    optimizer = Adam(learning_rate=0.00002, clipnorm=1.0)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [10]:
if train_model:
    history = model.fit(X, y, batch_size=32, epochs=50)

    model_save_path = '/content/model-saveLSTM.h5'
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    plt.plot(history.history['loss'])
    plt.title('Model Loss Over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    loss_plot_path = '/content/model_loss_plot.png'
    plt.savefig(loss_plot_path)
    print(f"Loss plot saved to {loss_plot_path}")
    plt.show()


Epoch 1/50


KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense

# Define the initial learning rate
initial_learning_rate = 0.002

# Define a scheduler function that multiplies the learning rate by 0.97 after each epoch
def scheduler(epoch, lr):
    if epoch < 1:  # For the first epoch, return the initial learning rate
        return lr
    else:
        return lr * 0.97  # Multiply the learning rate by the decay rate after each epoch

# Create a LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(scheduler)

# Assuming you have already defined seq_length, total_chars and load_saved_model
if load_saved_model:
    model = load_model('/content/model-saveLSTM.h5')
else:
    model = Sequential()
    model.add(LSTM(256, input_shape=(seq_length, total_chars), return_sequences=True))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(total_chars, activation='softmax'))

    # Create an Adam optimizer with the initial learning rate
    optimizer = Adam(learning_rate=initial_learning_rate, clipnorm=1.0)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Now, when you fit your model, make sure to include the callback
# For example:
# model.fit(x_train, y_train, epochs=50, callbacks=[lr_scheduler])
if train_model:
    history = model.fit(X, y, batch_size=32, epochs=50, callbacks=[lr_scheduler])

    model_save_path = '/content/model-saveLSTM-parameters.h5'
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    plt.plot(history.history['loss'])
    plt.title('Model Loss Over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    loss_plot_path = '/content/model_loss_plot.png'
    plt.savefig(loss_plot_path)
    print(f"Loss plot saved to {loss_plot_path}")
    plt.show()

Epoch 1/50
24434/24434 [==============================] - 385s 15ms/step - loss: 1.8299 - lr: 0.0020
Epoch 2/50
24434/24434 [==============================] - 373s 15ms/step - loss: 1.4548 - lr: 0.0019
Epoch 3/50
24434/24434 [==============================] - 371s 15ms/step - loss: 1.3573 - lr: 0.0019
Epoch 4/50
24434/24434 [==============================] - 360s 15ms/step - loss: 1.2918 - lr: 0.0018
Epoch 5/50
24434/24434 [==============================] - 359s 15ms/step - loss: 1.2399 - lr: 0.0018
Epoch 6/50
24434/24434 [==============================] - 359s 15ms/step - loss: 1.1976 - lr: 0.0017
Epoch 7/50
24434/24434 [==============================] - 359s 15ms/step - loss: 1.1616 - lr: 0.0017
Epoch 8/50
24434/24434 [==============================] - 358s 15ms/step - loss: 1.1303 - lr: 0.0016
Epoch 9/50
24434/24434 [==============================] - 357s 15ms/step - loss: 1.1002 - lr: 0.0016
Epoch 10/50
24434/24434 [==============================] - 357s 15ms/step - loss: 1.0723 - 

In [ ]:
generated_text = generate_text(200, 4)
print(generated_text)

with open('LSTM-output.txt', 'w') as file:
    file.write(generate_text(1000, 0.5))
